In [ ]:
# Install LangChain and required dependencies if not already installed
!pip install langchain openai
!pip install -U langchain-openai!pip install langchain langchain-community openai

# LangChain Agent with Multiple Tools
This notebook demonstrates how to build an AI agent using LangChain that has :
- Check the weather ,Get the latest news,Solve math problems,Tell a joke

In [ ]:
from langchain.agents import Tool, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType
import random
import httpx
from openai import OpenAI
import os

In [ ]:
model_key = "Enter_your_API_KEY"

In [ ]:
model_api="https://llama-3-1-8b-instruct-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com:443/v1"
model="meta-llama/Llama-3.1-8B-Instruct"


In [3]:
# Define functions for each tool
def get_weather(city: str) -> str:
    return f"The current weather in {city} is {random.randint(25, 35)}°C with partly cloudy skies."

def get_news(topic: str) -> str:
    return f"Top headlines on {topic}: 1. News One 2. News Two 3. News Three"

def get_joke(_: str = "") -> str:
    return "Why don't scientists trust atoms? Because they make up everything!"

def get_math_answer(expression: str) -> str:
    try:
        return f"The result is: {eval(expression)}"
    except:
        return "Sorry, couldn't calculate that."

In [19]:
# Create tool list
tools = [
    Tool(name="WeatherTool", func=get_weather, description="Useful for getting the weather for a city. Input should be a city name.",return_direct=True),
    Tool(name="NewsTool", func=get_news, description="Useful for getting latest news headlines about a topic. Input should be a topic name.",return_direct=True),
    Tool(name="JokeTool", func=get_joke, description="Useful for telling a random joke.",return_direct=True),
    Tool(name="MathTool", func=get_math_answer, description="Useful for solving simple math expressions. Input should be a valid math expression.",return_direct=True)
]

In [14]:
# Setup LLM and agent

llm = ChatOpenAI(
    model=model,
    temperature=0,
    base_url=model_api,
    api_key=model_key,
)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True,max_iterations=3)

In [23]:
query = "Make me laugh"
print("🧠 Agent Response:", agent.run(query))



> Entering new AgentExecutor chain...
Thought: I need to find a way to make the user laugh. I have a tool that can tell a random joke.

Action: JokeTool
Action Input: 
Observation: Why don't scientists trust atoms? Because they make up everything!


> Finished chain.
🧠 Agent Response: Why don't scientists trust atoms? Because they make up everything!


In [24]:
query = "What kind of outfit I need to wear for the climate in Kashmir"
raw_response = agent.run(query)

print("🧠 Agent Response:", raw_response)



> Entering new AgentExecutor chain...
Thought: To determine the outfit needed for the climate in Kashmir, I need to know the current weather conditions there.
Action: WeatherTool
Action Input: Kashmir
Observation: The current weather in Kashmir is 27°C with partly cloudy skies.


> Finished chain.
🧠 Agent Response: The current weather in Kashmir is 27°C with partly cloudy skies.


In [ ]:
query = "What kind of outfit I need to wear for the climate in Kashmir"
raw_response = agent.run(query)

print("🧠 Agent Response:", raw_response)

summary_prompt = (
    f"User asked: '{query}'\n"
    f"The tool gave this info: '{raw_response}'\n"
    "Based on this, provide a helpful information which is relevant to what user asked."
)

client = OpenAI(
    base_url=model_api,
    api_key=model_key,
    http_client=httpx.Client(verify=False)  # DO NOT DO THIS IN PRODUCTION!!!!
)

summary_response = client.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": summary_prompt}]
)

final_output = summary_response.choices[0].message.content.strip()
print("🧥 Final Summarized Response:", final_output)




> Entering new AgentExecutor chain...
Thought: To determine the outfit needed for the climate in Kashmir, I need to know the current weather conditions there.
Action: WeatherTool
Action Input: Kashmir
Observation: The current weather in Kashmir is 28°C with partly cloudy skies.


> Finished chain.
🧠 Agent Response: The current weather in Kashmir is 28°C with partly cloudy skies.
🧥 Final Summarized Response: Considering the temperature of 28°C with partly cloudy skies in Kashmir, I'd recommend dressing in light and breathable clothing to cater to the warm weather. 

Here are some outfit suggestions:

1. Lightweight tops: Opt for cotton, linen, or silk tops that allow for airflow and keep you cool. 
2. Loose-fitting pants: Synthetic or cotton pants would be a good choice for Kashmir's climate. You can also consider lightweight pants with a bit of stretch for comfort.
3. Comfortable shoes: Wear sandals, sneakers, or lightweight boots that are suitable for walking.
4. Add a light jacket:

In [29]:
# Run multiple queries
queries = [
    "What's the weather like in Chennai?",
    "Tell me the latest news about cricket.",
    "Can you solve 12 * (5 + 3)?",
    "I need a good joke."
]

for query in queries:
    print(f"\n🔍 Query: {query}")
    print("🧠 Agent Response:", agent.run(query))


🔍 Query: What's the weather like in Chennai?


> Entering new AgentExecutor chain...
Thought: I need to get the current weather for Chennai.
Action: WeatherTool
Action Input: Chennai
Observation: The current weather in Chennai is 35°C with partly cloudy skies.


> Finished chain.
🧠 Agent Response: The current weather in Chennai is 35°C with partly cloudy skies.

🔍 Query: Tell me the latest news about cricket.


> Entering new AgentExecutor chain...
Thought: I need to find the latest news about cricket.
Action: NewsTool
Action Input: cricket
Observation: Top headlines on cricket: 1. News One 2. News Two 3. News Three


> Finished chain.
🧠 Agent Response: Top headlines on cricket: 1. News One 2. News Two 3. News Three

🔍 Query: Can you solve 12 * (5 + 3)?


> Entering new AgentExecutor chain...
Thought: To solve the math expression, I need to follow the order of operations (PEMDAS/BODMAS), which means I should first evaluate the expression inside the parentheses.
Action: MathTool
Action